# Loading Data & Tools

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import re
import requests
from tqdm import tqdm

link = 'https://data.london.gov.uk/download/animal-rescue-incidents-attended-by-lfb/01007433-55c2-4b8a-b799-626d9e3bc284/Animal%20Rescue%20incidents%20attended%20by%20LFB%20from%20Jan%202009.csv'

In [ ]:
data = pd.read_csv(link)

In [ ]:
data.drop(columns = ['TypeOfIncident', 'SpecialServiceType',
                     'UPRN', 'USRN',
                     'Easting_m', 'Northing_m',
                     'Easting_rounded', 'Northing_rounded'], inplace = True)

# Custom Functions

### Info2

In [ ]:
def info2(df,h=1000):

  import plotly.graph_objects as go

  header = ['column name', 'dtype', 'entries', 'missing', 'missing %','unique values', 'most freaquent', 'sample value']
  dtype = [str(x) for x in list(df.dtypes)]
  missing = list(df.isnull().sum())
  per = list((df.isnull().mean() * 100).round(2))
  count = list(df.count().values)
  unique = list(df.nunique())
  sample = list(df.sample(n=1, random_state=98).copy().iloc[0])
  mf = []
  for x in df.columns:
    try:
      y = df[x].value_counts().idxmax()
      mf.append(y)
    except:
      mf.append(np.nan)

  info_frame=pd.DataFrame(columns=header)

  info_frame['column name'] = df.columns
  info_frame['dtype'] = dtype
  info_frame['entries'] = count
  info_frame['missing'] = missing
  info_frame['missing %'] = per
  info_frame['unique values'] = unique
  info_frame['most freaquent'] = mf
  info_frame['sample value'] = sample
  # print('Table shape:', df.shape[0], 'rows and', df.shape[1], 'columns\n')

  # del header, missing, per, count, sample, #info_frame

  info_table = go.Figure(data=[go.Table(
    header=dict(values=list(info_frame.columns),
                line_color='ivory',
                fill_color='darkgrey',
                align='left',
                # font=dict(color='ivory')
                ),
    cells=dict(values= info_frame.values.T.tolist(), # list of lists where each list is a column
               line_color='#eeeedd',
               fill_color='ivory',
               align='left',
              #  font=dict(color='#4d1919')
               ))])

  info_table.update_layout(title='Information about the dataset', height=h)
  info_table.show()

  #printing table summary
  print(f"\t   Table Information:\n\t   - {df.shape[0]} rows\n\t   - {df.shape[1]} columns\n\t   - {df.size} cells\n")

  y = info_frame.shape[0]
  x = info_frame[info_frame['missing %'] > 0].shape[0]
  c = list(info_frame.loc[info_frame['missing %'] > 0,'column name'])

  print(f"\t   {x} or {int(x/y*100)}% out of {y} columns have missing values.\n\n\t   Columns with missing values: \n\t   {c}")
  print(f"\t   \n\t   {df.isnull().sum().sum()} missing values or {round(df.isnull().sum().sum() / df.size * 100,2)}% of all values.\n")

  if df.duplicated().sum() != 0:
    if df.duplicated().sum() == 1:
      print(f"\t   {df.duplicated().sum()} duplicated row.\n")
    else:
      print(f"\t   {df.duplicated().sum()} duplicated rows.\n")
  else:
    print("\t   No duplicated rows.\n")

  return info_frame


## Symbols look-up

In [ ]:
#function to look for unallowed symbols

def symbol_lookup(df, column, allowed_symbols):
    pattern = r"[^a-zA-Z0-9" + re.escape(allowed_symbols) + "]"
    unique_values = df[column].unique()
    unallowed_symbols = set()

    for value in unique_values:
        symbols = re.findall(pattern, str(value))
        unallowed_symbols.update(symbols)
    if len(unallowed_symbols) == 0:
      print("No unallowed symbols found")
    else:
      print(unallowed_symbols)
      return list(unallowed_symbols)

## X-Ray

In [ ]:
#X-RAY
def xray(df):
  # Create a dataframe from the dictionary
  xray = px.imshow(data.isnull())
  xray.update_layout(title="Missing Values X-Ray", coloraxis_showscale=False, ) #width=1000, height=600
  # xray.show()

  dups = pd.DataFrame({f'Column_{i+1}': data.duplicated() for i in range(len(data.columns))})
  for x in dups.columns:
      dups[x] = dups[x].astype(int)

  dups = px.imshow(dups)
  dups.update_layout(title="X-Ray - Duplicated Rows", coloraxis_showscale=False)
  # dups.show()

  fig = make_subplots(rows=1, cols=2, subplot_titles=("Missing Values", "Duplicated Rows"))

  # Add plots to the subplots
  fig.add_trace(xray.data[0], row=1, col=1)
  fig.add_trace(dups.data[0], row=1, col=2)

  # Update the layout
  fig.update_layout(coloraxis_showscale=False, width=1400, height=600)

  #print number and % of missing values
  print('\nX-Ray Results:')
  print(f"  - {data.isnull().sum().sum()} missing values or {round(data.isnull().sum().sum() / data.size * 100,2)}% of all values.")

  #print number of duplicated rows
  if data.duplicated().sum() != 0:
    if data.duplicated().sum() == 1:
      print(f"   {data.duplicated().sum()} duplicated row.")
    else:
      print(f"  - {data.duplicated().sum()} duplicated rows.")
  else:
    print("  - No duplicated rows.")

  fig.show()

## Column check

In [ ]:
def col_check(name):
  col_name = name
  col = data[col_name]


  print(f"{col_name}: {col.dtypes}\n")

  s = col.describe()
  c = int(s['count'])
  u = col.nunique()
  try:
    t = s['top']
    f = s['freq']
  except:
    t = col.mode()
    f = col.value_counts().max()

  print("- missing values:", col.isnull().sum())
  print("- total values:", c)
  print("- unique values:", u)
  if c == u:
    print(f"- all values are unique")
  else:
    print("- not all values are unique")

  try:
    print(f"- min: {int(col.min())}")
    print(f"- max: {int(col.max())}")
  except:
    pass

  print(f"- most frequent: {t} ({f} times)")

  print("\nSample:\n", col.sample(n=5).to_list(),'\n')

# Quick Info

In [ ]:
xray(data)
info_frame = info2(data,830)


X-Ray Results:
  - 10266 missing values or 4.67% of all values.
  - No duplicated rows.


	   Table Information:
	   - 9565 rows
	   - 23 columns
	   - 219995 cells

	   10 or 43% out of 23 columns have missing values.

	   Columns with missing values: 
	   ['PumpCount', 'PumpHoursTotal', 'IncidentNotionalCost(£)', 'FinalDescription', 'WardCode', 'Ward', 'BoroughCode', 'Borough', 'Latitude', 'Longitude']
	   
	   10266 missing values or 4.67% of all values.

	   No duplicated rows.



In [ ]:
data.dtypes.value_counts()

object     18
float64     5
int64       2
dtype: int64

# Column diagnostics


## Incident number

In [ ]:
column = 'IncidentNumber'

col_check(column)

symbol_lookup(data, column, '0123456789-')

IncidentNumber: object

- missing values: 0
- total values: 9565
- unique values: 9565
- all values are unique
- most frequent: 139091 (1 times)

Sample:
 ['205721111', '081149-03072020', '087313-16072020', '035450-12032023', '045922-19042016'] 

No unallowed symbols found


In [ ]:
#rows where "-" is not in IncidentNumber
print(data[data['IncidentNumber'].str.contains('-') == False].shape[0], 'rows where "-" is not in IncidentNumber')

4066 rows where "-" is not in IncidentNumber


In [ ]:
print(data[column].duplicated().sum(), 'duplicated values.')


0 duplicated values.


To do:
`Recommendations`:
- rename column to 'ID'
- reset ID values for incidents that do not match the main format
- each incident number is generated from time stamp


## DateTimeOfCall

In [ ]:
column = 'DateTimeOfCall'
col_check(column)

symbol_lookup(data, column, '0123456789-: ')

DateTimeOfCall: object

- missing values: 0
- total values: 9565
- unique values: 9552
- not all values are unique
- most frequent: 2022-10-17 13:28:00 (2 times)

Sample:
 ['2018-08-02 09:19:00', '2016-10-25 19:07:00', '2021-03-11 13:05:00', '2017-05-10 12:48:00', '2013-07-10 19:56:00'] 

No unallowed symbols found


To do:
- converted to `datetime`
- split into separate `date` and `time` columns

## CalYear

In [ ]:
#def function for integer columns stats - count, unique, min , max, mode, freq
column = 'CalYear'
col_check(column)

#plotly histogram
fig = px.violin(data[column], title=f'{column} Histogram', width=500, height=500)
fig.update_layout(showlegend=False)
fig.show()

CalYear: int64

- missing values: 0
- total values: 9565
- unique values: 15
- not all values are unique
- min: 2009
- max: 2023
- most frequent: 2022 (1029 times)

Sample:
 [2011, 2019, 2012, 2022, 2015] 



## FinYear

In [ ]:
column = "FinYear"
col_check(column)

symbol_lookup(data, column, '0123456789/')

#plotly histogram
fig = px.histogram(data[column], title=f'{column} Histogram', nbins=data[column].nunique(), width=data[column].nunique()*44, height=data[column].nunique()*30, text_auto=True)
fig.update_layout(showlegend=False)
fig.show()

FinYear: object

- missing values: 0
- total values: 9565
- unique values: 16
- not all values are unique
- most frequent: 2022/23 (1036 times)

Sample:
 ['2021/22', '2012/13', '2011/12', '2016/17', '2023/24'] 

No unallowed symbols found


To do:
- convert to category

## PumpCount

In [ ]:
column = "PumpCount"
col_check(column)

#plotly histogram
fig = px.histogram(data[column], title=f'{column} Histogram', nbins=data[column].nunique(), width=500, height=400, text_auto=True)
fig.update_layout(showlegend=False)
fig.show()

PumpCount: float64

- missing values: 65
- total values: 9500
- unique values: 4
- not all values are unique
- min: 1
- max: 4
- most frequent: 1 (9341 times)

Sample:
 [1.0, 1.0, 1.0, 1.0, 1.0] 



To do:
- convert to `integer`
- restore missing values

## PumpHoursTotal

In [ ]:
column = 'PumpHoursTotal'
col_check(column)

#plotly histogram
fig = px.histogram(data[column], title=f'{column} Histogram', nbins=data[column].nunique(), width=500, height=400, text_auto=True)
fig.update_layout(showlegend=False)
fig.show()

PumpHoursTotal: float64

- missing values: 66
- total values: 9499
- unique values: 12
- not all values are unique
- min: 0
- max: 12
- most frequent: 1 (8403 times)

Sample:
 [1.0, 1.0, 1.0, 1.0, 1.0] 



To do:
- convert to `integer`
- restore missing values
- rename to `PumpHours`

## HourlyNotionalCost(£)

In [ ]:
column = 'HourlyNotionalCost(£)'
col_check(column)

#plotly histogram
fig = px.violin(data[column], title=f'{column}', width=500, height=500)
fig.update_layout(showlegend=False)
fig.show()

HourlyNotionalCost(£): int64

- missing values: 0
- total values: 9565
- unique values: 13
- not all values are unique
- min: 255
- max: 388
- most frequent: 260 (2391 times)

Sample:
 [352, 295, 352, 260, 260] 



To do:
- rename to`RatePerHour`
- convert to `integer`

## IncidentNotionalCost(£)

In [ ]:
column = 'IncidentNotionalCost(£)'
col_check(column)

#plotly histogram
fig = px.violin(data[column], title=f'{column}', width=500, height=500)
fig.update_layout(showlegend=False)
fig.show()

IncidentNotionalCost(£): float64

- missing values: 66
- total values: 9499
- unique values: 82
- not all values are unique
- min: 0
- max: 3912
- most frequent: 260 (2095 times)

Sample:
 [364.0, 704.0, 260.0, 328.0, 346.0] 



To do:
- rename to `IncidentCost`
- convert to `integer`
- restore missing values

## FinalDescription

In [ ]:
column = 'FinalDescription'
col_check(column)

FinalDescription: object

- missing values: 5
- total values: 9560
- unique values: 5796
- not all values are unique
- most frequent: Redacted (1372 times)

Sample:
 ['ASSIST RSPCA', 'ASSIST RSPCA WITH CAT STUCK BEHIND A WALL', 'Redacted', 'HAMSTER STUCK IN CAVITY WALL', 'DOG IN RIVER'] 



To do:
- convert to `string`
- replace `Redacted` with `-`
- restore missing values
- equalize to title/lower case format

## AnimalGroupParent

In [ ]:
column = 'AnimalGroupParent'
col_check(column)

symbol_lookup(data, column, 'abcdefghijklmnopqrstuvwxyz- ')

#plotly histogram
fig = px.histogram(data[column], title=f'{column} Histogram', nbins=data[column].nunique(), width=800, height=800, text_auto=True)
fig.update_layout(showlegend=False)
fig.show()

AnimalGroupParent: object

- missing values: 0
- total values: 9565
- unique values: 28
- not all values are unique
- most frequent: Cat (4708 times)

Sample:
 ['Dog', 'Dog', 'Dog', 'Unknown - Domestic Animal Or Pet', 'Cat'] 

No unallowed symbols found


In [ ]:
data[column].value_counts(sort=True)

Cat                                                        4708
Bird                                                       1933
Dog                                                        1437
Fox                                                         506
Unknown - Domestic Animal Or Pet                            224
Horse                                                       212
Deer                                                        164
Unknown - Wild Animal                                       107
Squirrel                                                     84
Unknown - Heavy Livestock Animal                             51
cat                                                          24
Hamster                                                      20
Snake                                                        20
Rabbit                                                       19
Ferret                                                       10
Cow                                     

To do:
- convert to 'category'
- rename to `Animal`
- equalize to title or lower case format
- remove incident types from animal type

- replace values:

  1. `Unknown - Domestic Animal Or Pet`
    - > with *Other Domestic Animal*

  1. `Unknown - Wild Animal`
    - > with *Other Wild Animal*

  1. `Unknown - Heavy Livestock Animal`

  1. `Unknown - Animal rescue from water - Farm animal`

  1. `Unknown - Animal rescue from below ground - Farm animal`
    - > with *Other Farm Animal*

## OriginofCall

In [ ]:
column = 'OriginofCall'
col_check(column)

#plotly histogram
fig = px.histogram(data[column], title=f'{column} Histogram', nbins=data[column].nunique(), width=500, height=500, text_auto=True)
fig.update_layout(showlegend=False)
fig.show()

OriginofCall: object

- missing values: 0
- total values: 9565
- unique values: 8
- not all values are unique
- most frequent: Person (mobile) (5880 times)

Sample:
 ['Person (mobile)', 'Person (land line)', 'Person (mobile)', 'Person (land line)', 'Person (mobile)'] 



To do:
- rename to `OriginOfCall`
- replace values:

  > `Person (land line)`
  
  > `Person (mobile)`

  > `Person (running call)`

  > `Not known`
  
    - with '*Person*'

  > `Other FRS`
  - with *'Fire Brigade'*


## PropertyType

In [ ]:
column = 'PropertyType'
col_check(column)

# symbol_lookup(data, column, 'abcdefghijklmnopqrstuvwxyz -')

PropertyType: object

- missing values: 0
- total values: 9565
- unique values: 187
- not all values are unique
- most frequent: House - single occupancy  (2498 times)

Sample:
 ['Other car park structure', 'Purpose Built Flats/Maisonettes - Up to 3 storeys ', 'Canal/riverbank vegetation ', 'House - single occupancy ', 'Animal harm outdoors'] 



In [ ]:
data[['PropertyCategory',column]].sample(n=20)

,PropertyCategory,PropertyType
3452,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...
4430,Dwelling,House - single occupancy
6437,Outdoor,Domestic garden (vegetation not equipment)
4121,Non Residential,Private garage
6535,Outdoor Structure,Bridge
1891,Dwelling,House - single occupancy
3154,Dwelling,House - single occupancy
2283,Dwelling,House - single occupancy
9028,Other Residential,Other Residential Home
815,Dwelling,House - single occupancy


## PropertyCategory

In [ ]:
column = 'PropertyCategory'
col_check(column)

symbol_lookup(data, column, 'abcdefghijklmnopqrstuvwxyz ')

#plotly histogram
fig = px.histogram(data[column], title=f'{column} Histogram', nbins=data[column].nunique(), width=500, height=500, text_auto=True)
fig.update_layout(showlegend=False)
fig.show()

PropertyCategory: object

- missing values: 0
- total values: 9565
- unique values: 7
- not all values are unique
- most frequent: Dwelling (5010 times)

Sample:
 ['Outdoor', 'Outdoor', 'Outdoor', 'Dwelling', 'Outdoor'] 

No unallowed symbols found


To do:
- convert to category

## SpecialServiceTypeCategory

In [ ]:
column = 'SpecialServiceTypeCategory'
col_check(column)

#plotly histogram
fig = px.histogram(data[column], title=f'{column} Histogram', nbins=data[column].nunique(), width=500, height=500, text_auto=True)
fig.update_layout(showlegend=False)
fig.show()

SpecialServiceTypeCategory: object

- missing values: 0
- total values: 9565
- unique values: 4
- not all values are unique
- most frequent: Other animal assistance (4576 times)

Sample:
 ['Other animal assistance', 'Other animal assistance', 'Other animal assistance', 'Other animal assistance', 'Other animal assistance'] 



To do:
- convert to category

## WardCode

In [ ]:
column = 'WardCode'
col_check(column)

symbol_lookup(data, column, '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')

print(f"\nValues format UPPER case:")
try:
  print(f"True:  {data[data[column].isnull() == False][column].apply(str.isupper).value_counts()[True]}\nFalse: {data[column].apply(str.isupper).value_counts()[False]}")
except:
  print(f"True:  {data[data[column].isnull() == False][column].apply(str.isupper).value_counts()[True]}\nFalse: 0")

WardCode: object

- missing values: 10
- total values: 9555
- unique values: 759
- not all values are unique
- most frequent: E05013971 (47 times)

Sample:
 ['E05013792', 'E05013908', 'E05011466', 'E05013784', 'E05013713'] 

No unallowed symbols found

Values format UPPER case:
True:  9555
False: 0


To do:
- convert to category
- restore missing values


## Ward

In [ ]:
column = 'Ward'
col_check(column)

symbol_lookup(data, column, "abcdefghijklmnopqrstuvwxyz&.' ")

print(f"\nValues format lower case:")
print(data[data[column].isnull() == False][column].apply(str.islower).value_counts())

print(f"\nValues format UPPER case:")
print(data[data[column].isnull() == False][column].apply(str.isupper).value_counts())

print(f"\nValues format Title case:")
print(data[data[column].isnull() == False][column].apply(str.istitle).value_counts())


Ward: object

- missing values: 10
- total values: 9555
- unique values: 1243
- not all values are unique
- most frequent: Gooshays (42 times)

Sample:
 ['Central Greenford', 'Norbiton', 'Colham & Cowley', "ST. DUNSTAN'S", 'Hornsey'] 

{',', '-'}

Values format lower case:
False    9555
Name: Ward, dtype: int64

Values format UPPER case:
False    7815
True     1740
Name: Ward, dtype: int64

Values format Title case:
True     7398
False    2157
Name: Ward, dtype: int64


In [ ]:
#the following Wards need a fix
wards1 = data[(data[column].isnull() == False) & (data[column].str.contains('-'))][column]
wards2 = data[(data[column].isnull() == False) & (data[column].str.contains(','))][column]
wards1.append(wards2)


<ipython-input-60-24884d43b28b>:3: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



1219                     Havering-atte-Bower
4670                     Havering-atte-Bower
5063                     Havering-atte-Bower
5197                     Havering-atte-Bower
5339                     Havering-atte-Bower
5767                     Havering-atte-Bower
6204                     Havering-atte-Bower
6544                     Havering-atte-Bower
7199                     Havering-atte-Bower
8092                     Havering-atte-Bower
8184                     Havering-atte-Bower
8569                     HAVERING-ATTE-BOWER
9108                     HAVERING-ATTE-BOWER
9236                     HAVERING-ATTE-BOWER
9479                     HAVERING-ATTE-BOWER
38       Ham, Petersham & Richmond Riverside
123      Ham, Petersham & Richmond Riverside
125      Ham, Petersham & Richmond Riverside
253      Ham, Petersham & Richmond Riverside
572      Ham, Petersham & Richmond Riverside
629      Ham, Petersham & Richmond Riverside
631      Ham, Petersham & Richmond Riverside
652     Mo

To do:
- convert to category
- equalize case format
- restore missing values
- fix wrong values

## BoroughCode

In [ ]:
column = 'BoroughCode'
col_check(column)

symbol_lookup(data, column, "abcdefghijklmnopqrstuvwxyz0123456789")

print(f"\nValues format lower case:")
print(data[data[column].isnull() == False][column].apply(str.islower).value_counts())

print(f"\nValues format UPPER case:")
print(data[data[column].isnull() == False][column].apply(str.isupper).value_counts())

print(f"\nValues format Title case:")
print(data[data[column].isnull() == False][column].apply(str.istitle).value_counts())


BoroughCode: object

- missing values: 12
- total values: 9553
- unique values: 37
- not all values are unique
- most frequent: E09000010 (439 times)

Sample:
 ['E09000010', 'E09000032', 'E09000016', 'E09000009', 'E09000027'] 

No unallowed symbols found

Values format lower case:
False    9553
Name: BoroughCode, dtype: int64

Values format UPPER case:
True    9553
Name: BoroughCode, dtype: int64

Values format Title case:
True    9553
Name: BoroughCode, dtype: int64


To do:
- convert to category
- restore missing values

## Borough

In [ ]:
column = 'Borough'
col_check(column)

symbol_lookup(data, column, "abcdefghijklmnopqrstuvwxyz& ")

print(f"\nValues format lower case:")
print(data[data[column].isnull() == False][column].apply(str.islower).value_counts())

print(f"\nValues format UPPER case:")
print(data[data[column].isnull() == False][column].apply(str.isupper).value_counts())

print(f"\nValues format Title case:")
print(data[data[column].isnull() == False][column].apply(str.istitle).value_counts())


Borough: object

- missing values: 12
- total values: 9553
- unique values: 70
- not all values are unique
- most frequent: ENFIELD (275 times)

Sample:
 ['RICHMOND UPON THAMES', 'GREENWICH', 'HARINGEY', 'Lambeth', 'GREENWICH'] 

No unallowed symbols found

Values format lower case:
False    9553
Name: Borough, dtype: int64

Values format UPPER case:
True     5618
False    3935
Name: Borough, dtype: int64

Values format Title case:
False    6104
True     3449
Name: Borough, dtype: int64


To do:
- convert to category
- restore missing values
- equalize case format

## StnGroundName

In [ ]:
column = 'StnGroundName'
col_check(column)

symbol_lookup(data, column, "abcdefghijklmnopqrstuvwxyz& ")

print(f"\nValues format lower case:")
print(data[data[column].isnull() == False][column].apply(str.islower).value_counts())

print(f"\nValues format UPPER case:")
print(data[data[column].isnull() == False][column].apply(str.isupper).value_counts())

print(f"\nValues format Title case:")
print(data[data[column].isnull() == False][column].apply(str.istitle).value_counts())

StnGroundName: object

- missing values: 0
- total values: 9565
- unique values: 108
- not all values are unique
- most frequent: Enfield (188 times)

Sample:
 ['Euston', 'Clapham', 'New Malden', 'Shoreditch', 'East Greenwich'] 

No unallowed symbols found

Values format lower case:
False    9565
Name: StnGroundName, dtype: int64

Values format UPPER case:
False    9565
Name: StnGroundName, dtype: int64

Values format Title case:
True    9565
Name: StnGroundName, dtype: int64


To do:
- convert to category

## PostcodeDistrict

In [ ]:
column = 'PostcodeDistrict'
col_check(column)

symbol_lookup(data, column, "abcdefghijklmnopqrstuvwxyz& ")

print(f"\nValues format lower case:")
print(data[data[column].isnull() == False][column].apply(str.islower).value_counts())

print(f"\nValues format UPPER case:")
print(data[data[column].isnull() == False][column].apply(str.isupper).value_counts())

print(f"\nValues format Title case:")
print(data[data[column].isnull() == False][column].apply(str.istitle).value_counts())


PostcodeDistrict: object

- missing values: 0
- total values: 9565
- unique values: 277
- not all values are unique
- most frequent: CR0 (169 times)

Sample:
 ['W13', 'IG11', 'W3', 'BR2', 'N12'] 

No unallowed symbols found

Values format lower case:
False    9565
Name: PostcodeDistrict, dtype: int64

Values format UPPER case:
True    9565
Name: PostcodeDistrict, dtype: int64

Values format Title case:
False    6556
True     3009
Name: PostcodeDistrict, dtype: int64


To do:
- equalize case format
- convert to category

## Street

In [ ]:
column = 'Street'
col_check(column)

symbol_lookup(data, column, "abcdefghijklmnopqrstuvwxyz. '")

print(f"\nValues format lower case:")
print(data[data[column].isnull() == False][column].apply(str.islower).value_counts())

print(f"\nValues format UPPER case:")
print(data[data[column].isnull() == False][column].apply(str.isupper).value_counts())

print(f"\nValues format Title case:")
print(data[data[column].isnull() == False][column].apply(str.istitle).value_counts())

Street: object

- missing values: 0
- total values: 9565
- unique values: 7076
- not all values are unique
- most frequent: HIGH STREET (44 times)

Sample:
 ['PENWITH ROAD', 'Sherrard Road', 'BARNLEA CLOSE', 'TASH PLACE', 'MARTINS ROAD'] 

{')', '(', '-', '/'}

Values format lower case:
False    9565
Name: Street, dtype: int64

Values format UPPER case:
True     5630
False    3935
Name: Street, dtype: int64

Values format Title case:
False    5655
True     3910
Name: Street, dtype: int64


In [ ]:
streets = data[data[column].str.contains('\)')][column].to_list()
streets.extend(data[data[column].str.contains('/')][column].to_list())
streets.extend(data[data[column].str.contains('-')][column].to_list())
streets

['SOUTH TERRACE FOOTPATH (OPP 7 SOUTH TERRACE-SOUTH BANK CARPARK R/O SURBITON STATION)',
 'LONG LANE SERVICE ROAD O/S HILLINGDON STATION',
 'SOUTH TERRACE FOOTPATH (OPP 7 SOUTH TERRACE-SOUTH BANK CARPARK R/O SURBITON STATION)',
 'FOOTPATH F/O 23-28 FOXGLOVE LANE',
 'SOUTH TERRACE FOOTPATH (OPP 7 SOUTH TERRACE-SOUTH BANK CARPARK R/O SURBITON STATION)',
 'FOOTPATH F/O 23-28 FOXGLOVE LANE',
 'UNNAMED FOOTPATH - MILL ROAD TO PERCY ROAD',
 'UNNAMED ROAD - RICHMOND PARK - HAM GATE TO PARK',
 'UNNAMED FOOTPATH - LONSDALE ROAD TO HAMMERSMITH BRIDGE',
 'GREAT SOUTH-WEST ROAD']

To do:
- equalize case format
- convert to category
- fix street names containing extra info, or not

## Latitude & Longitude

In [ ]:
column = 'Latitude'
col_check(column)


Latitude: float64

- missing values: 5010
- total values: 4555
- unique values: 4485
- not all values are unique
- min: 0
- max: 51
- most frequent: 0    51.608305
1    51.636755
Name: Latitude, dtype: float64 (4 times)

Sample:
 [51.4571138198, nan, nan, nan, nan] 



In [ ]:
column = 'Longitude'
col_check(column)


Longitude: float64

- missing values: 5010
- total values: 4555
- unique values: 4485
- not all values are unique
- min: 0
- max: 0
- most frequent: 0   -0.033228
1   -0.016520
Name: Longitude, dtype: float64 (4 times)

Sample:
 [-0.4423946655, -0.1704462111, nan, -0.1125136011, nan] 



In [ ]:
fig = px.scatter(data, x='Longitude', y='Latitude', title='Locations', width=800, height=800, opacity=.5, color='BoroughCode',)
fig.update_layout(showlegend=False)
fig.update_yaxes(type='log')
fig.show()

To do:
- restore missing values

# Fixes

## rename columns

In [ ]:
print(f"column names:\n{list(data.columns)}\n\nRenaming columns...\n")

#rename columns AnimalGroupParent, PumpHoursTotal, OriginofCall
names = {'IncidentNumber':'ID',
         'CalYear':'Year',
         'HourlyNotionalCost(£)': 'HourlyCost',
         'IncidentNotionalCost(£)': 'IncidentCost',
         'FinalDescription': 'Description',
         'AnimalGroupParent': 'Animal',
         'PumpHoursTotal': 'PumpHours',
         'OriginofCall': 'OriginOfCall',
         'SpecialServiceTypeCategory': 'OperationType',
         }

data.rename(columns=names, inplace=True)

print(f"New column names:\n{list(data.columns)}")

column names:
['IncidentNumber', 'DateTimeOfCall', 'CalYear', 'FinYear', 'PumpCount', 'PumpHoursTotal', 'HourlyNotionalCost(£)', 'IncidentNotionalCost(£)', 'FinalDescription', 'AnimalGroupParent', 'OriginofCall', 'PropertyType', 'PropertyCategory', 'SpecialServiceTypeCategory', 'WardCode', 'Ward', 'BoroughCode', 'Borough', 'StnGroundName', 'Street', 'PostcodeDistrict', 'Latitude', 'Longitude']

Renaming columns...

New column names:
['ID', 'DateTimeOfCall', 'Year', 'FinYear', 'PumpCount', 'PumpHours', 'HourlyCost', 'IncidentCost', 'Description', 'Animal', 'OriginOfCall', 'PropertyType', 'PropertyCategory', 'OperationType', 'WardCode', 'Ward', 'BoroughCode', 'Borough', 'StnGroundName', 'Street', 'PostcodeDistrict', 'Latitude', 'Longitude']


## strings case formats

### title case

In [ ]:
#convert strings in Description, Animal, OriginOfCall, PropertyType, PropertyCategory, OperationType, Ward, Borough, StnGroundName, Street to title letter case
columns = ['Description', 'Animal', 'OriginOfCall', 'PropertyType', 'PropertyCategory', 'OperationType', 'Ward', 'Borough', 'StnGroundName', 'Street']
data[columns] = data[columns].apply(lambda x: x.str.title())
data[columns].sample(n=3)

,Description,Animal,OriginOfCall,PropertyType,PropertyCategory,OperationType,Ward,Borough,StnGroundName,Street
4298,Redacted,Cat,Person (Land Line),Converted Flat/Maisonettes - 3 Or More Storeys,Dwelling,Other Animal Assistance,Coningham,Hammersmith And Fulham,Hammersmith,St Stephen'S Avenue
6189,Dog Stuck On Roof,Dog,Person (Mobile),Purpose Built Flats/Maisonettes - Up To 3 Stor...,Dwelling,Animal Rescue From Height,Rushey Green,Lewisham,Lewisham,Rushey Green
5055,Bird Loose In Lounge,Bird,Person (Land Line),House - Single Occupancy,Dwelling,Other Animal Assistance,Trinity,Wandsworth,Tooting,Beechcroft Road


### upper case

In [ ]:
#convert WardCode, BoroughCode, PostcodeDistrict to capital letter case
columns = ['WardCode', 'BoroughCode', 'PostcodeDistrict']
data[columns] = data[columns].apply(lambda x: x.str.upper())
data[columns].sample(n=7)

,WardCode,BoroughCode,PostcodeDistrict
4438,E05013659,E09000007,NW3
3057,E05013529,E09000009,W7
269,E05013725,E09000023,SE13
3347,E05011102,E09000028,SE17
7355,E05013921,E09000025,E12
4656,E05013535,E09000009,W5
4079,E05013577,E09000017,HA4


## DateTimeOfCall

In [ ]:
#DateTimeOfCall to datetime
data['DateTimeOfCall'] = pd.to_datetime(data['DateTimeOfCall'])

In [ ]:
# date
data['Date'] = data['DateTimeOfCall'].dt.date

# HourOfCall
data['HourOfCall'] = data['DateTimeOfCall'].dt.hour

# week#
data['WeekNumber'] = data['DateTimeOfCall'].dt.isocalendar().week

# WeekDay
data['WeekDay'] = data['DateTimeOfCall'].dt.dayofweek+1

#get day of week (Mon, Tue, Wed, Thu, Fri, Sat, Sun)
data['DayOfWeek'] = data['DateTimeOfCall'].dt.day_name()

# del data['DateTimeOfCall']
data.columns


Index(['ID', 'DateTimeOfCall', 'Year', 'FinYear', 'PumpCount', 'PumpHours',
       'HourlyCost', 'IncidentCost', 'Description', 'Animal', 'OriginOfCall',
       'PropertyType', 'PropertyCategory', 'OperationType', 'WardCode', 'Ward',
       'BoroughCode', 'Borough', 'StnGroundName', 'Street', 'PostcodeDistrict',
       'Latitude', 'Longitude', 'Date', 'HourOfCall', 'WeekNumber', 'WeekDay',
       'DayOfWeek'],
      dtype='object')

In [ ]:
str(data['DateTimeOfCall'][0]).replace('-', '').replace(':', '').replace(' ', '')

'20090101030100'

## ID

In [ ]:
#function to create ID from date and time

def id_creator(df, id_col, date, time):
  for i in range(len(df)):
    id = str(df['DateTimeOfCall'][i]).replace('-', '').replace(':', '').replace(' ', '')
    df.loc[i, id_col] = id

id_creator(data, 'ID', 'Date', 'Time')
data.ID

0       20090101030100
1       20090101085100
2       20090104100700
3       20090105122700
4       20090106152300
             ...      
9560    20230530001300
9561    20230530204100
9562    20230530231600
9563    20230531101000
9564    20230531122400
Name: ID, Length: 9565, dtype: object

In [ ]:
symbol_lookup(data, 'ID', '0123456789')

No unallowed symbols found


## Animal

In [ ]:
#replace values in Animal column:
# Unknown - Domestic Animal Or Pet with Other Domestic Animal
data['Animal'] = data['Animal'].replace(['Unknown - Domestic Animal Or Pet'], 'Other Domestic Animal')


# Unknown - Wild Animal with Other Wild Animal
data['Animal'] = data['Animal'].replace(['Unknown - Wild Animal'], 'Other Wild Animal')

# Unknown - Heavy Livestock Animal, Unknown - Animal Rescue From Water - Farm Animal, Unknown - Animal Rescue From Below Ground - Farm Animal with Other Farm Animal
data['Animal'] = data['Animal'].replace(['Unknown - Heavy Livestock Animal',
                                         'Unknown - Animal Rescue From Water - Farm Animal',
                                         'Unknown - Animal Rescue From Below Ground - Farm Animal'], 'Other Farm Animal')

data['Animal'].unique()


array(['Dog', 'Fox', 'Horse', 'Rabbit', 'Other Farm Animal', 'Squirrel',
       'Cat', 'Bird', 'Other Domestic Animal', 'Sheep', 'Deer',
       'Other Wild Animal', 'Snake', 'Lizard', 'Hedgehog', 'Hamster',
       'Lamb', 'Fish', 'Bull', 'Cow', 'Ferret', 'Budgie', 'Pigeon',
       'Goat', 'Tortoise'], dtype=object)

## Description

In [ ]:
data['Description'].value_counts()

Redacted                                         1372
Small Animal Rescue                               169
Bird Trapped In Chimney                            94
Cat Stuck On Roof                                  67
Bird Trapped In Netting                            60
                                                 ... 
Bird Stuck On Lamp Post - Hanging From String       1
Dog Trapped In Quaggy                               1
Possibly Injured Cat On Stuck Roof                  1
Distressed Dog Locked In Car                        1
Fox Trapped - Caller On Scene                       1
Name: Description, Length: 5796, dtype: int64

In [ ]:
#replace Redacted values in Description with -
data['Description'] = data['Description'].replace(['Redacted'], np.nan)
data.isnull().sum()

ID                     0
DateTimeOfCall         0
Year                   0
FinYear                0
PumpCount             65
PumpHours             66
HourlyCost             0
IncidentCost          66
Description         1377
Animal                 0
OriginOfCall           0
PropertyType           0
PropertyCategory       0
OperationType          0
WardCode              10
Ward                  10
BoroughCode           12
Borough               12
StnGroundName          0
Street                 0
PostcodeDistrict       0
Latitude            5010
Longitude           5010
Date                   0
HourOfCall             0
WeekNumber             0
WeekDay                0
DayOfWeek              0
dtype: int64

## OriginOfCall

In [ ]:
print(list(data['OriginOfCall'].unique()))
#replace values in OriginOfCall:
# 'Person (land line)', 'Person (Mobile)', 'Person (running call)', 'Not known' with 'Person'
data['OriginOfCall'] = data['OriginOfCall'].replace(['Person (Land Line)', 'Person (Mobile)', 'Person (Running Call)', 'Not Known'], 'Person')
# 'Other FRS' with 'The Fire Brigade'
data['OriginOfCall'] = data['OriginOfCall'].replace(['Other Frs'], 'Fire Brigade')

#value counts
print(list(data['OriginOfCall'].unique()))

['Person (Land Line)', 'Person (Mobile)', 'Ambulance', 'Police', 'Other Frs', 'Person (Running Call)', 'Coastguard', 'Not Known']
['Person', 'Ambulance', 'Police', 'Fire Brigade', 'Coastguard']


## missing values

In [ ]:
data.isnull().sum()

ID                     0
DateTimeOfCall         0
Year                   0
FinYear                0
PumpCount             65
PumpHours             66
HourlyCost             0
IncidentCost          66
Description         1377
Animal                 0
OriginOfCall           0
PropertyType           0
PropertyCategory       0
OperationType          0
WardCode              10
Ward                  10
BoroughCode           12
Borough               12
StnGroundName          0
Street                 0
PostcodeDistrict       0
Latitude            5010
Longitude           5010
Date                   0
HourOfCall             0
WeekNumber             0
WeekDay                0
DayOfWeek              0
dtype: int64

### PumpCount & PumpHours

In [ ]:
miss1 = ['PumpCount', 'PumpHours', 'IncidentCost']
data[data['PumpHours'].isnull()][miss1]

,PumpCount,PumpHours,IncidentCost
314,NaN,NaN,NaN
355,NaN,NaN,NaN
523,NaN,NaN,NaN
1268,NaN,NaN,NaN
1394,NaN,NaN,NaN
...,...,...,...
8834,NaN,NaN,NaN
8882,NaN,NaN,NaN
8977,NaN,NaN,NaN
9054,NaN,NaN,NaN


Each of the columns `PumpCount`, `PumpHours` and `IncidentCost` have missing values at the same rows.

In [ ]:
#plot plotly express histogram of PumpHours and PumpCount
fig1 = px.histogram(data['PumpHours'], title='PumpHours distribution', width=500, height=400, text_auto=True)
fig1.update_layout(showlegend=False)
fig1.show()

# % distribution of PumpHours
print(data['PumpHours'].value_counts()/data.shape[0]*100)

1.0     87.851542
2.0      7.997909
3.0      2.300052
4.0      0.554104
5.0      0.313643
7.0      0.073183
6.0      0.073183
9.0      0.062729
8.0      0.031364
12.0     0.020910
0.0      0.020910
10.0     0.010455
Name: PumpHours, dtype: float64


In [ ]:
#plot plotly express histogram of PumpHours and PumpCount
fig1 = px.histogram(data['PumpCount'], title='PumpCount distribution', width=500, height=400, text_auto=True)
fig1.update_layout(showlegend=False)
fig1.show()
print(data['PumpCount'].value_counts()/data.shape[0]*100)

1.0    97.658129
2.0     1.547308
3.0     0.104548
4.0     0.010455
Name: PumpCount, dtype: float64


In [ ]:
data[['PumpCount','PumpHours']].value_counts()/data.shape[0]*100

PumpCount  PumpHours
1.0        1.0          87.799268
           2.0           7.987454
           3.0           1.348667
2.0        3.0           0.951385
1.0        4.0           0.386827
2.0        5.0           0.219550
           4.0           0.156822
1.0        5.0           0.094093
2.0        7.0           0.062729
           1.0           0.052274
           6.0           0.041819
3.0        9.0           0.031364
           6.0           0.031364
2.0        9.0           0.020910
1.0        0.0           0.020910
2.0        8.0           0.020910
           10.0          0.010455
           12.0          0.010455
3.0        2.0           0.010455
           7.0           0.010455
           8.0           0.010455
1.0        9.0           0.010455
3.0        12.0          0.010455
4.0        4.0           0.010455
dtype: float64

In [ ]:
# Identify missing values in PumpCount column
missing_indices = data[data['PumpCount'].isnull()].index

# Calculate the number of missing values to assign 1 to based on the desired percentage
X = 97
num_to_assign = int(X / 100 * len(missing_indices))

# Randomly select indices to assign 1
indices_to_assign = np.random.choice(missing_indices, size=num_to_assign, replace=False)

# Assign 1 to the selected missing values
data.loc[indices_to_assign, 'PumpCount'] = 1

# Assign 2 to the rest
data.loc[data['PumpCount'].isnull(),'PumpCount'] = 2

In [ ]:
missing_indices = data[data['PumpHours'].isnull()].index
num_to_assign = int(87 / 100 * len(missing_indices))
indices_to_assign = np.random.choice(missing_indices, size=num_to_assign, replace=False)

data.loc[indices_to_assign, 'PumpHours'] = 1

missing_indices = data[data['PumpHours'].isnull()].index
num_to_assign = int(90 / 100 * len(missing_indices))
indices_to_assign = np.random.choice(missing_indices, size=num_to_assign, replace=False)

data.loc[indices_to_assign, 'PumpHours'] = 2
data.loc[data['PumpHours'].isnull(),'PumpHours'] = 3

In [ ]:
data.isnull().sum()

ID                     0
Year                   0
FinYear                0
PumpCount              0
PumpHours              0
HourlyCost             0
IncidentCost          66
Description         1377
Animal                 0
OriginOfCall           0
PropertyType           0
PropertyCategory       0
OperationType          0
WardCode              10
Ward                  10
BoroughCode           12
Borough               12
StnGroundName          0
Street                 0
PostcodeDistrict       0
Latitude            5010
Longitude           5010
Date                   0
Time                   0
WeekNumber             0
WeekDay                0
DayOfWeek              0
dtype: int64

### IncidentCost

In [ ]:
data[data['PumpCount']>1][['PumpHours','PumpCount','HourlyCost','IncidentCost']].sample(n=7)

,PumpHours,PumpCount,HourlyCost,IncidentCost
3927,3.0,2.0,298,894.0
185,7.0,3.0,260,1820.0
8088,5.0,2.0,352,1760.0
260,3.0,2.0,260,780.0
9020,5.0,2.0,364,1820.0
1209,3.0,2.0,260,780.0
1431,4.0,2.0,260,1040.0


In [ ]:
#IncidentCost = PumpHours * HourlyCost (this reflects the price for 1 pump per hour)
costs = data[data['IncidentCost'].isnull()][['PumpHours','HourlyCost']]
data.loc[data['IncidentCost'].isnull(),'IncidentCost'] = costs['PumpHours']*costs['HourlyCost']
data.isnull().sum()

ID                     0
DateTimeOfCall         0
Year                   0
FinYear                0
PumpCount              0
PumpHours              0
HourlyCost             0
IncidentCost           0
Description         1377
Animal                 0
OriginOfCall           0
PropertyType           0
PropertyCategory       0
OperationType          0
WardCode              10
Ward                  10
BoroughCode           12
Borough               12
StnGroundName          0
Street                 0
PostcodeDistrict       0
Latitude            5010
Longitude           5010
Date                   0
HourOfCall             0
WeekNumber             0
WeekDay                0
DayOfWeek              0
dtype: int64

### Borough, BoroughCode, Ward, WardCode

In [ ]:
columns = ['DateTimeOfCall','WardCode','Ward','BoroughCode','Borough','StnGroundName','Street','PostcodeDistrict','Latitude','Longitude']
data[data['Borough'].isnull()][columns]



,DateTimeOfCall,WardCode,Ward,BoroughCode,Borough,StnGroundName,Street,PostcodeDistrict,Latitude,Longitude
4137,2016-01-25 18:03:00,NaN,NaN,NaN,NaN,Essex,COOLGARDIE AVENUE,IG7,51.618437,0.067940
4164,2016-02-24 21:03:00,NaN,NaN,NaN,NaN,Essex,PALMERSTON ROAD,IG9,51.627354,0.037281
4245,2016-04-28 15:00:00,NaN,NaN,NaN,NaN,Essex,BRADWELL ROAD,IG9,51.631296,0.057254
4708,2016-12-26 13:30:00,E05013885,Chingford Green,NaN,NaN,Chingford,RANGERS ROAD,IG9,51.634787,0.019796
5896,2019-01-30 11:26:00,NaN,NaN,NaN,NaN,Kent,MEAD WALL,ME3,51.471845,0.466421
6338,2019-09-23 18:44:00,NaN,NaN,NaN,NaN,Buckinghamshire,GEORGE GREEN ROAD,SL3,51.520078,-0.559730
6767,2020-06-10 13:56:00,E05013936,King George's & Sunray,NaN,NaN,Surbiton,GATLEY AVENUE,KT19,51.365708,-0.282855
7071,2020-10-02 12:38:00,NaN,NaN,NaN,NaN,Surrey,BRAMLEY ROAD,SM3,51.351315,-0.228279
7477,2021-05-09 09:46:00,E05013570,Heathrow Villages,NaN,NaN,Heathrow,SOUTHERN PERIMETER ROAD,TW6,51.462070,-0.486104
7655,2021-06-23 00:28:00,NaN,NaN,NaN,NaN,Essex,CHURCH ROAD,RM14,51.547928,0.359629


In [ ]:
#Filling missing data for rows with ID
combo1= ['Borough','BoroughCode','Ward','WardCode']
combo2= ['Borough','BoroughCode']
#row ID 4137 - Borough: Epping Forest , bCode: E07000072  , Ward: Chigwell Village , wCode: E05004151
data.loc[4137, combo1] = ['Epping Forest', 'E07000072', 'Chigwell Village', 'E05004151']

#row ID 4164 - Borough: Epping Forest , bCode: E07000072 , Ward: Buckhurst Hill West , wCode: E05004149
data.loc[4164, combo1] = ['Epping Forest', 'E07000072', 'Buckhurst Hill West', 'E05004149']

#row ID 4245 - Borough: Epping Forest , bCode: E07000072 , Ward: Buckhurst Hill East , wCode: E05004148
data.loc[4245, combo1] = ['Epping Forest', 'E07000072', 'Buckhurst Hill East', 'E05004148']

#row ID 4708 - Borough: Waltham Forest , bCode: E09000031
data.loc[4708, combo2] = ['Waltham Forest', 'E09000031']

#row ID 5896 - Borough: Medway , bCode: E06000035 , Ward: Strood Rural  , wCode: E05002266
data.loc[5896, combo1] = ['Medway', 'E06000035', 'Strood Rural', 'E05002266']

#row ID 6338 - Borough: Buckinghamshire , bCode: E06000060 , Ward: Stoke Poges and Wexham , wCode: E05013158
data.loc[6338, combo1] = ['Buckinghamshire', 'E06000060', 'Stoke Poges and Wexham', 'E05013158']

#row ID 6767 - Borough: Kingston upon Thames , bCode: E09000021
data.loc[6767, combo2] = ['Kingston upon Thames', 'E09000021']

#row ID 7071 - Borough: Epsom and Ewell , bCode: E07000208, Ward: Nonsuch , wCode: E05007279
data.loc[7071, combo1] = ['Epsom and Ewell', 'E07000208', 'Nonsuch', 'E05007279']

#row ID 7680 - Borough: Epsom and Ewell, bCode: E07000208 , Ward: College , wCode: E05007274
data.loc[7680, combo1] = ['Epsom and Ewell', 'E07000208', 'College', 'E05007274']

#row ID 7477 - Borough: Hillingdon , bCode: E09000017
data.loc[7477, combo2] = ['Hillingdon', 'E09000017']

#row ID 7655 - Borough: Thurrock , bCode: E06000034 , Ward: Orsett , wCode: E05002240
data.loc[7655, combo1] = ['Thurrock', 'E06000034', 'Orsett', 'E05002240']

#row ID 8975 - Borough: Slough , bCode: E06000039 , Ward: Colnbrook with Poyle, wCode: E05009343
data.loc[8975, combo1] = ['Slough', 'E06000039', 'Colnbrook with Poyle', 'E05009343']

#row ID 2304 - Ward: Rosedale and Bury Green , wCode: E05009009
data.loc[2304, ['Ward','WardCode']] = ['Rosedale and Bury Green', 'E05009009']

In [ ]:
data.isnull().sum()

ID                     0
DateTimeOfCall         0
Year                   0
FinYear                0
PumpCount              0
PumpHours              0
HourlyCost             0
IncidentCost           0
Description         1377
Animal                 0
OriginOfCall           0
PropertyType           0
PropertyCategory       0
OperationType          0
WardCode               0
Ward                   0
BoroughCode            0
Borough                0
StnGroundName          0
Street                 0
PostcodeDistrict       0
Latitude            5010
Longitude           5010
Date                   0
HourOfCall             0
WeekNumber             0
WeekDay                0
DayOfWeek              0
dtype: int64

### Latitude & Longitude

In [ ]:
data[data['Latitude'].isnull()][['Borough','BoroughCode','Ward','WardCode','Street','Latitude','Longitude']].sample(n=7)

,Borough,BoroughCode,Ward,WardCode,Street,Latitude,Longitude
6521,Havering,E09000016,Rush Green & Crowlands,E05013980,Spring Gardens,NaN,NaN
4664,Hillingdon,E09000017,Yeading,E05013583,Marsworth Close,NaN,NaN
6898,Merton,E09000024,Pollards Hill,E05013821,Chestnut Grove,NaN,NaN
4270,Richmond Upon Thames,E09000027,Mortlake & Barnes Common,E05013783,Clifford Avenue,NaN,NaN
3327,Haringey,E09000014,Hornsey,E05013593,Rathcoole Gardens,NaN,NaN
1147,Merton,E09000024,Lavender Fields,E05013817,Wilson Avenue,NaN,NaN
844,Ealing,E09000009,Southfield,E05013540,Somerset Road,NaN,NaN


In [ ]:
#extracting local data based on WardCode
def get_locs(code):
  link = f'https://findthatpostcode.uk/areas/{code}.json'
  response = requests.get(link)
  file_ = response.json()
  file_len = len(file_)
  locs = pd.DataFrame(columns=['WardCode','lat','lon'])

  if file_len == 0:
    message = 'No data available for'
    return message
  else:
    for i in range(file_len):
      if 'attributes' in file_['included'][i]:
        if 'location' in file_['included'][i]['attributes']:
          loc = file_['included'][i]['attributes']['location']
          #update locs
          locs.loc[i] = loc

  locs.rename(columns={'lat':'Latitude','lon':'Longitude'}, inplace=True)
  locs2 = data[data['WardCode'] == code][['Latitude', 'Longitude']]
  locs2 = locs2.dropna()

  locs = pd.concat([locs, locs2])
  locs['WardCode'] = code
  return locs


In [ ]:
def new_locs(df, code):
  locations_ = get_locs(code)
  missing = df[df['Latitude'].isnull()].loc[df['WardCode'] == code, ['Latitude', 'Longitude']].shape[0]
  new_locs_ = pd.DataFrame(columns=['Latitude','Longitude', 'WardCode'])
  available = new_locs_.shape[0]

  # get random location
  def random_loc():
    lat = locations_['Latitude'].sample(n=1).iloc[0]
    lon = locations_['Longitude'].sample(n=1).iloc[0]
    location = lat,lon
    return location

  # get mean location
  def mean_loc():
    lat,lon = locations_[['Latitude','Longitude']].mean()
    location = lat,lon
    return location

  while missing > available:
    loc = random_loc()
    if not new_locs_.loc[(new_locs_['Latitude'] == loc[0]) & (new_locs_['Longitude'] == loc[1])].empty:
        loc = mean_loc()
    new_locs_ = pd.concat([new_locs_, pd.DataFrame({'Latitude': [loc[0]], 'Longitude': [loc[1]]})], ignore_index=True)
    available += 1

  new_locs_['WardCode'] = locations_['WardCode'].unique()[0]
  return new_locs_

In [ ]:
def all_new_locs():
  new = pd.DataFrame(columns=['Latitude','Longitude', 'WardCode'])

  for x in tqdm(data[data['Latitude'].isnull()]['WardCode'].unique()):
    temp_locs = pd.DataFrame(columns=['Latitude','Longitude', 'WardCode'])
    try:
      temp_locs = new_locs(data, x)
      new = pd.concat([new, temp_locs])
    except:
      print('Problem with', x)

  return new

new_locations = all_new_locs()
new_locations

 63%|██████▎   | 426/679 [18:22<11:47,  2.80s/it]

Problem with E05000165


 82%|████████▏ | 558/679 [23:41<04:11,  2.08s/it]

Problem with E05000083


 89%|████████▉ | 606/679 [25:30<03:50,  3.15s/it]

Problem with E05000637


100%|██████████| 679/679 [28:22<00:00,  2.51s/it]

Problem with E05009290


,Latitude,Longitude,WardCode
0,51.421513,-0.087066,E05011467
1,51.417526,-0.087066,E05011467
2,51.419284,-0.094476,E05011467
3,51.421513,-0.083494,E05011467
4,51.417526,-0.081583,E05011467
...,...,...,...
0,51.482718,-0.138663,E05014015
0,51.568774,-0.279519,E05013497
0,51.546367,-0.007712,E05013925
0,51.557444,-0.285578,E05013516


In [ ]:
# new_locations.to_csv('new_locations.csv', index=False)
# data.to_csv('data.csv', index=False)

In [ ]:
# create new column with values True/False where True is Latitude is missing
data['Restored'] = data['Latitude'].isnull()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9565 entries, 0 to 9564
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                9565 non-null   object        
 1   DateTimeOfCall    9565 non-null   datetime64[ns]
 2   Year              9565 non-null   int64         
 3   FinYear           9565 non-null   object        
 4   PumpCount         9565 non-null   float64       
 5   PumpHours         9565 non-null   float64       
 6   HourlyCost        9565 non-null   int64         
 7   IncidentCost      9565 non-null   float64       
 8   Description       8188 non-null   object        
 9   Animal            9565 non-null   object        
 10  OriginOfCall      9565 non-null   object        
 11  PropertyType      9565 non-null   object        
 12  PropertyCategory  9565 non-null   object        
 13  OperationType     9565 non-null   object        
 14  WardCode          9565 n

In [ ]:
# new_locations = pd.read_csv('/content/new_locations.csv')
new_locations2 = new_locations.copy()

In [ ]:
def fill_missing_lat_long(df1, df2):
    # Create a copy of df1 to avoid modifying the original DataFrame
    df1_copy = df1.copy()

    # Get the indices of rows in df1 with missing Latitude
    missing_indices = df1_copy[df1_copy['Latitude'].isnull()].index

    # Iterate over the missing indices
    for index in missing_indices:
        # Get the WardCode for the missing row
        ward_code = df1_copy.loc[index, 'WardCode']
        # Check if there is a matching row in df2 with the same WardCode
        if df2[df2['WardCode'] == ward_code].index.size > 0:
          matching_row = df2[df2['WardCode'] == ward_code].sample(n=1)
          # If a matching row is found, fill the missing Latitude and Longitude values
          df1_copy.loc[index, 'Latitude'] = matching_row['Latitude'].values[0]
          df1_copy.loc[index, 'Longitude'] = matching_row['Longitude'].values[0]

          # Drop the row from df2
          df2.drop(matching_row.index, inplace=True)

    return df1_copy

new_data = fill_missing_lat_long(data, new_locations2)

In [ ]:
new_data.isnull().sum()

ID                     0
DateTimeOfCall         0
Year                   0
FinYear                0
PumpCount              0
PumpHours              0
HourlyCost             0
IncidentCost           0
Description         1377
Animal                 0
OriginOfCall           0
PropertyType           0
PropertyCategory       0
OperationType          0
WardCode               0
Ward                   0
BoroughCode            0
Borough                0
StnGroundName          0
Street                 0
PostcodeDistrict       0
Latitude               4
Longitude              4
Date                   0
HourOfCall             0
WeekNumber             0
WeekDay                0
DayOfWeek              0
Restored               0
dtype: int64

In [ ]:
new_data[new_data['Latitude'].isnull()]

,ID,DateTimeOfCall,Year,FinYear,PumpCount,PumpHours,HourlyCost,IncidentCost,Description,Animal,...,WardCode,Ward,BoroughCode,Borough,StnGroundName,Street,PostcodeDistrict,Latitude,Longitude,Restored
1674,161580111,2011-10-03 20:23:00,2011,2011/12,1.0,1.0,260,260.0,Cat Stuck Between House And Draining Area,Cat,...,E05000165,Thornton Heath,E09000008,Croydon,Norbury,Grange Road,CR7,NaN,NaN,True
3099,36130141,2014-03-25 10:13:00,2014,2013/14,1.0,1.0,290,290.0,Running Call To Bird Trapped Behind Fire Place,Bird,...,E05000083,Sidcup,E09000004,Bexley,Sidcup,Manor Road,DA15,NaN,NaN,True
3807,75521151,2015-06-16 12:43:00,2015,2015/16,1.0,1.0,298,298.0,Parrot Trapped In Excavation,Bird,...,E05000637,Knightsbridge And Belgravia,E09000033,Westminster,Chelsea,Ennismore Gardens Mews,SW7,NaN,NaN,True
9553,075296-28052023,2023-05-28 17:00:00,2023,2023/24,1.0,1.0,388,388.0,Crows Stuck Behind Balcony Caller Will Meet...,Bird,...,E05009290,Bassishaw,E09000001,City Of London,Shoreditch,Wood Street,EC2Y,NaN,NaN,True


In [ ]:
#assign new values to new_data
# index 1674, Latitude 51.34799, Longitude -0.10034
new_data.loc[1674, ['Latitude', 'Longitude']] = [51.34799, -0.10034]

# index 3099, Latitude, 51.43211 Longitude  0.10086
new_data.loc[3099, ['Latitude', 'Longitude']] = [51.43211, 0.10086]

# index 3807, Latitude, 51.49917 Longitude -0.17171
new_data.loc[3807, ['Latitude', 'Longitude']] = [51.49917, -0.17171]

# index 9553, Latitude 51.51670, Longitude -0.09378
new_data.loc[9553, ['Latitude', 'Longitude']] = [51.51670, -0.09378]

new_data.isnull().sum()

ID                     0
DateTimeOfCall         0
Year                   0
FinYear                0
PumpCount              0
PumpHours              0
HourlyCost             0
IncidentCost           0
Description         1377
Animal                 0
OriginOfCall           0
PropertyType           0
PropertyCategory       0
OperationType          0
WardCode               0
Ward                   0
BoroughCode            0
Borough                0
StnGroundName          0
Street                 0
PostcodeDistrict       0
Latitude               0
Longitude              0
Date                   0
HourOfCall             0
WeekNumber             0
WeekDay                0
DayOfWeek              0
Restored               0
dtype: int64

## data types

In [ ]:
new_data.dtypes

ID                   object
Year                  int64
FinYear              object
PumpCount           float64
PumpHours           float64
HourlyCost            int64
IncidentCost        float64
Description          object
Animal               object
OriginOfCall         object
PropertyType         object
PropertyCategory     object
OperationType        object
WardCode             object
Ward                 object
BoroughCode          object
Borough              object
StnGroundName        object
Street               object
PostcodeDistrict     object
Latitude            float64
Longitude           float64
Date                 object
Time                 object
WeekNumber           UInt32
WeekDay               int64
DayOfWeek            object
Restored               bool
dtype: object

In [ ]:
#PumpCount, PumpHours, IncidentCost to int64
#Replace missing or infinite values with 0
cols = ['PumpCount', 'PumpHours', 'IncidentCost', 'WeekNumber']
# new_data[['PumpCount', 'PumpHours', 'IncidentCost']] = new_data[['PumpCount', 'PumpHours', 'IncidentCost']].replace([np.inf, -np.inf, np.nan], 0)
new_data[cols] = new_data[cols].astype('int64')

#Animal, OriginOfCall, PropertyCategory, OperationType, WardCode, Ward, Borough, BoroughCode, StnGroundName, PostcodeDistrict to categorical
cols = ['Animal','OriginOfCall','PropertyCategory', 'OperationType',
        'WardCode', 'Ward', 'Borough', 'BoroughCode', 'StnGroundName', 'PostcodeDistrict',
        'FinYear', 'DayOfWeek']
new_data[cols] = new_data[cols].astype('category')

new_data['Date'] = pd.to_datetime(new_data['Date'])


new_data.dtypes

ID                          object
DateTimeOfCall      datetime64[ns]
Year                         int64
FinYear                   category
PumpCount                    int64
PumpHours                    int64
HourlyCost                   int64
IncidentCost                 int64
Description                 object
Animal                    category
OriginOfCall              category
PropertyType                object
PropertyCategory          category
OperationType             category
WardCode                  category
Ward                      category
BoroughCode               category
Borough                   category
StnGroundName             category
Street                      object
PostcodeDistrict          category
Latitude                   float64
Longitude                  float64
Date                datetime64[ns]
HourOfCall                   int64
WeekNumber                   int64
WeekDay                      int64
DayOfWeek                 category
Restored            

## TrueCost

In [ ]:
new_data['TrueCost'] = new_data['IncidentCost'] * new_data['PumpCount']

# Final Table

In [ ]:
col_order = ['ID', 'DateTimeOfCall', 'Year', 'FinYear', 'Date', 'HourOfCall', 'WeekNumber', 'WeekDay', 'DayOfWeek',
             'StnGroundName', 'OperationType', 'Animal',
             'PumpCount', 'PumpHours', 'HourlyCost', 'IncidentCost', 'TrueCost',
             'PostcodeDistrict', 'Borough', 'BoroughCode', 'Ward', 'WardCode', 'Street', 'Latitude', 'Longitude',
             'PropertyType', 'PropertyCategory',
             'OriginOfCall', 'Description','Restored'
             ]
data = new_data[col_order]

In [ ]:
info_frame = info2(data, 910)

	   Table Information:
	   - 9565 rows
	   - 30 columns
	   - 286950 cells

	   1 or 3% out of 30 columns have missing values.

	   Columns with missing values: 
	   ['Description']
	   
	   1377 missing values or 0.48% of all values.

	   No duplicated rows.



In [ ]:
data.sample(n=22)

,ID,DateTimeOfCall,Year,FinYear,Date,HourOfCall,WeekNumber,WeekDay,DayOfWeek,StnGroundName,...,Ward,WardCode,Street,Latitude,Longitude,PropertyType,PropertyCategory,OriginOfCall,Description,Restored
4534,20160909072500,2016-09-09 07:25:00,2016,2016/17,2016-09-09,7,36,5,Friday,Holloway,...,Tollington,E05013712,Thorpedale Road,51.565317,-0.121725,House - Single Occupancy,Dwelling,Person,Fox Trapped In Gate,True
7264,20210131125300,2021-01-31 12:53:00,2021,2020/21,2021-01-31,12,4,7,Sunday,Enfield,...,Whitewebbs,E05013695,Brodie Road,51.666770,-0.086153,Domestic Garden (Vegetation Not Equipment),Outdoor,Person,Cat Stuck Up Tree - Rspca On Scene,False
6415,20191116171300,2019-11-16 17:13:00,2019,2019/20,2019-11-16,17,46,6,Saturday,Stratford,...,Forest Gate South,E05013912,Faraday Road,51.543306,0.013018,Fence,Outdoor Structure,Person,Cat Stuck On High Fence Assistance Requeste...,False
6578,20200323111600,2020-03-23 11:16:00,2020,2019/20,2020-03-23,11,13,1,Monday,Addington,...,Shirly South,E05011483,Broom Road,51.359528,-0.051462,Purpose Built Flats/Maisonettes - 4 To 9 Storeys,Dwelling,Person,Bird Caught On Netting Caught In Netting Fi...,True
445,20090916145300,2009-09-16 14:53:00,2009,2009/10,2009-09-16,14,38,3,Wednesday,Paddington,...,Regent'S Park,E05013805,Allitsen Road,51.525149,-0.164980,Purpose Built Flats/Maisonettes - Up To 3 Stor...,Dwelling,Person,Dog Trapped In Window Frame,True
2156,20120728202200,2012-07-28 20:22:00,2012,2012/13,2012-07-28,20,30,6,Saturday,Tottenham,...,Tottenham Hale,E05013602,Reed Road,51.590091,-0.061175,Purpose Built Flats/Maisonettes - Up To 3 Stor...,Dwelling,Person,Cat Trapped Behind Sink,True
5575,20180711163500,2018-07-11 16:35:00,2018,2018/19,2018-07-11,16,28,3,Wednesday,Wandsworth,...,Roehampton,E05014017,Roehampton High Street,51.450647,-0.238955,Single Shop,Non Residential,Person,Cat Trapped In Air Vent,False
7402,20210412183300,2021-04-12 18:33:00,2021,2021/22,2021-04-12,18,15,1,Monday,Stanmore,...,Kenton East,E05013551,Honeypot Lane,51.585533,-0.282754,Large Supermarket,Non Residential,Person,NaN,False
4934,20170528094900,2017-05-28 09:49:00,2017,2017/18,2017-05-28,9,21,7,Sunday,Euston,...,St. Pancras & Somers Town,E05013669,Chalton Street,51.537812,-0.138948,Purpose Built Flats/Maisonettes - Up To 3 Stor...,Dwelling,Person,Cat Stuck On Balcony On First Floor Possibly F...,True
8935,20220928175600,2022-09-28 17:56:00,2022,2022/23,2022-09-28,17,39,3,Wednesday,Wembley,...,Perivale,E05013534,Torrington Road,51.532051,-0.325426,House - Single Occupancy,Dwelling,Person,NaN,True


In [ ]:
data.to_csv('LFB - Animal Rescue Data.csv', index=False)